In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector

# this is taken fro!pip install mysql-connector-pythonm the github code for forming a connection to the database
cnx = mysql.connector.connect(user='umls', password='umls', host='172.16.34.1', port='3307', database='umls2019')

cursor = cnx.cursor()

# find circlesdatabases that is the 
query = '''
    SELECT  c1.str AS node1, c2.str AS node2, c3.str AS node3
    FROM MRCONSO c1
    JOIN MRREL r1 ON c1.cui = r1.cui1
    JOIN MRCONSO c2 ON r1.cui2 = c2.cui
    JOIN MRREL r2 ON c2.cui = r2.cui1
    JOIN MRCONSO c3 ON r2.cui2 = c3.cui
    JOIN MRREL r3 ON c3.cui = r3.cui1
    JOIN MRCONSO c4 ON r3.cui2 = c4.cui AND c4.TS = 'P'
    WHERE c1.tty = 'MSH' AND c2.tty = 'MSH' AND c3.tty = 'MSH' AND c4.tty = 'MSH'
    AND c1.str != c2.str AND c2.str != c3.str AND c3.str != c1.str AND c1.str != c4.str
    AND c2.str != c4.str AND c3.str != c4.str AND c1.str < c2.str AND c2.str < c3.str
    AND c1.cui != c2.cui AND c2.cui != c3.cui AND c3.cui != c1.cui AND c1.cui != c4.cui
    AND c2.cui != c4.cui AND c3.cui != c4.cui AND c1.cui < c2.cui AND c2.cui < c3.cui
    AND c1.cui IN (
        SELECT cui1 FROM MRREL WHERE rel = 'isa' AND cui2 = 'C0010054' AND stype2 = 'PT'
'''

cursor.execute(query)

# get results
results = cursor.fetchall()

# find circles with at least 3 and less than 40 different nodes
circles = set()
for result in results:
    circle = sorted(result)
    if len(circle) >= 3 and len(circle) < 40:
        circles.add(tuple(circle))

# output the first 3 circles found
for i, circle in enumerate(circles):
    if i >= 3:
        break
    circle_str = " to ".join(circle) + " to " + circle[0]
    print("Circle {}: {}".format(i+1, circle_str))
    
# close cursor and connection
cursor.close()
cnx.close()
